## <a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog"></a> <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/1_Prepare_Data_Annotate_Images_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/1_Prepare_Data_Annotate_Images_Part1.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



## Automatic Face Image Annotation  ##

We start off the data preparation phase by pre-processing images in our dataset for the unmasked category.

We have a selection of facial images and we need to draw bounding boxes around them. We could do that manually using a tool such as [labelImg](https://github.com/tzutalin/labelImg) however that would be slow, tedious and unnecessary for our purposes.

Instead we will use a facial detection algorithm (Haar cascade) to automatically detect faces and generate an accompanying XML file for each image with the coordinates of the face. Now this method is far from foolproof as it only works properly with full frontal images of faces but its a good starting point.



### Import required libraries

In [101]:
!pip install lxml
!pip install opencv-python
from lxml import etree as ET

import os
import cv2
import numpy as np
import glob
from pathlib import Path
import shutil

from PIL import Image, ImageFile

### Prepare dataset ###

#### Set image directory ####

In [102]:
dataset_dir = "dataset/unmasked"

#### Download required dependency 

In [103]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml

In [104]:
path = os.getcwd()
filename = path + "/" + dataset_dir
face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface_improved.xml')
dataset_labels_folder = dataset_dir + "-labels"
dataset_labels_path = os.getcwd()+ "/"+ dataset_labels_folder 
print(dataset_labels_path)
os.mkdir(dataset_labels_folder)


/home/jovyan/work/Generate_Data/dataset/unmasked-labels


In [105]:
for subdir, dirs, files in os.walk(filename):
    for file in files:

        
        img_path=os.path.join(subdir, file)
        img_name=os.path.basename(img_path)

        if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):

            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            h,w,bpp = np.shape(img)
            imgp = Image.open(img_path)
#            b, landmarks = detect_faces(imgp)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)


#            g=show_bboxes(imgp, b, landmarks)
            #print(len(faces))
            if len(faces) == 1 :
                try :

                    for bounding_boxes in faces:
                        face = img[int(bounding_boxes[1]):int(bounding_boxes[3]),
                        int(bounding_boxes[0]):int(bounding_boxes[2])]

                        
                        subdir_path, subdir_name = os.path.split(subdir)

                        root = ET.Element("annotation",verified="yes")
                        ET.SubElement(root, "folder").text=filename

                        ET.SubElement(root, "filename").text = img_name
                        ET.SubElement(root, "path").text = img_path

                        source=ET.SubElement(root, "source")
                        ET.SubElement(source, "database").text = "unknown"

                        size=ET.SubElement(root, "size")
                        ET.SubElement(size, "width").text = str(w)
                        ET.SubElement(size, "height").text = str(h)
                        ET.SubElement(size, "depth").text = str(bpp)

                        ET.SubElement(root, "segmented").text = "0"

                        obj=ET.SubElement(root, "object")
                        ET.SubElement(obj, "name").text = subdir_name
                        ET.SubElement(obj, "pose").text = "Unspecified"
                        ET.SubElement(obj, "truncated").text = "0"
                        ET.SubElement(obj, "difficult").text = "0"

                        box=ET.SubElement(obj, "bndbox")
                        ET.SubElement(box, "xmin").text = str(int(bounding_boxes[0]))
                        ET.SubElement(box, "ymin").text = str(int(bounding_boxes[1]))
                        ET.SubElement(box, "xmax").text = str(int(bounding_boxes[2])+int(bounding_boxes[1]))
                        ET.SubElement(box, "ymax").text = str(int(bounding_boxes[3]))

                        tree = ET.ElementTree(root)
                        tree.write(os.path.join(dataset_labels_path, os.path.splitext(img_name)[0] + '.xml'))
                        #tree.write(os.path.join(filename +"-labels", os.path.splitext(img_name)[0] + '.xml'))
                        
                except RuntimeError :
                    with open("delete.txt", "a") as myfile:
                        myfile.write(img_path+"\n")
                	#print(img_path)

                else :
                    with open("delete.txt", "a") as myfile:
                            myfile.write(img_path+"\n")
                    #print('no face')

In [106]:
print(dataset_labels_path)

/home/jovyan/work/Generate_Data/dataset/unmasked-labels


### Save Dataset

In [109]:
source1 = filename + "/"
source2 = dataset_labels_path + "/"

destination = os.getcwd() + "/training_demo"+ "/images"
if not os.path.exists(destination):
    os.makedirs(destination)


def move_files(source,destination):
    files = os.listdir(source)
    for f in files:
        shutil.move(source+f, destination)


move_files(source1,destination)
move_files(source2,destination)

# deletes source directories, use as needed
os.rmdir(source2)
os.rmdir(source1)

['6.jpg', '4.jpg', '1.jpg', '2.jpg', '3.jpg']
/home/jovyan/work/Generate_Data/dataset/unmasked/
/home/jovyan/work/Generate_Data/dataset/unmasked-labels/


### Compress Dataset (Optional) ###

Follow this step if you wish to archive your dataset for future use

In [ ]:
#!zip -r unmasked-dataset.zip dataset
#!tar -zcvf unmasked-dataset.tar.gz dataset